In [9]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
import torch.optim as optim
#import net
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20,kernel_size = 10)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
    
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = F.relu(self.fc2(x))    
        
        return F.log_softmax(x,dim=1)        

In [11]:
def train(args, model, device, train_loader, optimizer, opoch):
    model.train()
    train_loss = []
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(decvice), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.data.numpy())
        if batch_idx % args.log_interval == 0:
            print('Train Epoch:{} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
            epoch, batch_idx*len(data), len(train_loader.dataset),100. * batch_idx/len(train_loader),loss.item()))
    return sum(train_loss)/len(train_loss)

In [12]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device),target.to(device)
            output=model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy:{}/{} ({:.0f}%)\n'.format(
    test_loss, correct,len(test_loader.dataset), 100. *correct/len(test_loader.dataset)))

In [13]:
def main():
    parser = argparse.ArgumentParser(description='Pytorch MINIST Example')
    parser.add_argument('--batch_size', type=int, default=64, metavar='N',help='input batch size for training(default:64)')
    parser.add_argument('--test_batch_size', type=int, default=1000,metavar='N',help='input batch size for testing(default:1000)')
    parser.add_argument('--epochs', type=int, default=10, metavar='N', help='number of epochs to train(default:10)')
    parser.add_argument('--lr', type=float, default=0.001, metavar='LR', help='learning rate(default:0.001)')
    parser.add_argument('--momentum', type=float, default=0.5, metavar='M', help='SGD momentum(default:0.5)')
    parser.add_argument('--no-cuda', action='store_true',default=False, help='disables CUDA training')
    parser.add_argument('--seed', type=int, default=1, metavar='S', help='random seed(default:1)')
    parser.add_argument('--log-interval',type=int,default=10,metavar='N',help='how many batches to wait before logging training status')
    args = parser.parse_args()
    
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    
    torch.manual_seed(args.seed)
    
    device = torch.device("cuda" if use_cuda else "cpu")
    
    kwargs = {'num_workers':1, 'pin_memory':True} if use_cuda else{}
    
    train_dataset = datasets.MNIST('/data',train=True, download=True,
                                  transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.1307,),(0.3081,))
                                  ]))
    test_dataset = datasets.MNIST('./data',train=False,
                                 transform = transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.1307,),(0.3081,))
                                 ]))
    train_loader = torch.utils.data.DataLoader( train_dataset, batch_size=args.batch_size, shuffle=True,  **kwargs)
    test_loader = troch.utils.data.DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False,**kwargs)
    
    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(),lr=args.lr, momentum=args.momentum)
    
    writer = SummaryWriter('runs')
    
    for epoch in range(1,args.epochs+1):
        train_loss = train(args,model.device,train_loader,optimizer,epoch)
        writer.add_scalar('Train',train_loss,epoch)
        test(args,model,device,test_loader)
        
    writer.close()

In [14]:
if __name__ == '__main__':
    main()

usage: ipykernel_launcher.py [-h] [--batch_size N] [--test_batch_size N]
                             [--epochs N] [--lr LR] [--momentum M] [--no-cuda]
                             [--seed S] [--log-interval N]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-1d3ca4b3-fcfc-4bb8-8556-f0740e39d153.json


SystemExit: 2

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
